In [1]:
import os
import tempfile

from sklearn.metrics import accuracy_score
from PIL import Image


from aidefender.utils.mlflow import load_model, create_art_model, save_art_model
from aidefender.robustness import robustness_accuracy
from aidefender.defences.bart import BaRT
from aidefender.exp.datasets import create_dataset

# Declare constants

In [ ]:
dataset_class = 'aidefender.exp.datasets.RP2KDataset'
data_path = '../artifacts/data/RP2K_small/'
model_path = '../artifacts/models/mlflow_RP2K_pytorch/'

# Load data

In [ ]:
dataset = create_dataset(dataset_class, data_path)

In [ ]:
dataset.images.shape

In [ ]:
dataset.labels

## Show some images

In [ ]:
Image.fromarray(dataset.images[6])

In [ ]:
Image.fromarray(dataset.images[47])

# Load model

In [ ]:
mlflow_model = load_model(model_path)

In [ ]:
art_model = create_art_model(mlflow_model)

## Test model

In [ ]:
predictions = art_model.predict(dataset.images).argmax(-1)

In [ ]:
predictions

# Calc accuracy and robustness

In [ ]:
acc = accuracy_score(dataset.labels, predictions)

In [ ]:
acc

In [ ]:
robust_acc = robustness_accuracy(art_model, dataset.images, attack_name='fgsm')

In [ ]:
robust_acc

# Apply aidefender's BaRT defense

In [ ]:
defense = BaRT() # channels_first=art_model.channels_first

In [ ]:
defense

In [ ]:
art_model.set_params(preprocessing_defences=defense)

## Calc accuracy and robustness of the defended model

In [ ]:
predictions = art_model.predict(dataset.images).argmax(-1)

In [ ]:
acc = accuracy_score(dataset.labels, predictions)

In [ ]:
acc

In [ ]:
robust_acc = robustness_accuracy(art_model, dataset.images, attack_name='fgsm')

In [ ]:
robust_acc

# Save model

In [ ]:
tmp_dir = tempfile.mkdtemp(prefix='aidefender')

In [ ]:
tmp_dir

In [ ]:
model_path = os.path.join(tmp_dir, 'model')

In [ ]:
save_art_model(art_model, model_path)

In [ ]:
!cat {model_path}/MLmodel